### 1. Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense

import tf2onnx
import tensorflow as tf
import onnxruntime as rt

### 2. Load dataset

This is just a dummy example, loading a very simple and basic data.

In [ ]:
#Replace it based on cough recording training dataset
data_df = pd.read_csv('replace with input data')

X = data_df.iloc[:,:20].values 
y = data_df.iloc[:,20:21].values

### 3. Preprocessing

In [ ]:
def get_ohe(signal):
    ohe = OneHotEncoder()
    ohe_array = ohe.fit_transform(signal).toarray()
    return ohe_array

In [ ]:
y = get_ohe(y)

### 4. Train/Validation split

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size = 0.15)

### 5. Build Model

In [ ]:
def build_model():
    inputs = keras.layers.Input(shape=(20))
    
    x = keras.layers.Dense(32, activation="relu", name="dense_1")(inputs)
    x = keras.layers.Dense(16, activation="relu", name="dense_2")(x)
    output = keras.layers.Dense(4, activation="softmax", name="output")(x)
    
    model = keras.Model(inputs=inputs, outputs=output, name="tb_recognition")
    return model

In [ ]:
model = build_model()

#### 5.1 Compile

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 5.2 Checkpoints

In [ ]:
model_checkpoint = keras.callbacks.ModelCheckpoint("tb_cough_detection.h5", 
                                                      monitor="val_accuracy", 
                                                      save_best_only=True)
callbacks = [model_checkpoint]

#### 5.2 Fit model

In [ ]:
history = model.fit(train_X, train_y, 
                    validation_data = (val_X, val_y), 
                    epochs=5, batch_size=64, 
                    callbacks = callbacks)

### 6. Convert Keras to ONNX 

Converting Keras trained model into ONNX format model

In [ ]:
def keras_2_onnx():
    out_path = model.name + ".onnx"
    spec = (tf.TensorSpec((None, 20), tf.float32, name="input"),)
    m_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=out_path)
    
    out_name = [n.name for n in m_proto.graph.output]
    return out_name

In [ ]:
out_name = keras_2_onnx()

### 7. Prediction on ONNX model

Demo example to make prediction using ONNX format model

In [ ]:
def get_prediction(val_X):
    model_inference = rt.InferenceSession('tb_recognition.onnx')
    
    input_name = model_inference.get_inputs()[0].name
    label_name = model_inference.get_outputs()[0].name
    
    onnx_pred = model_inference.run([label_name], {input_name: val_X.astype(np.float32)})
    pred = np.argmax(onnx_pred[0], axis=1)
    return pred

In [ ]:
pred = get_prediction(val_X)